In [13]:
%run "../PyUtils/common.ipynb"
c = readFile("../PyUtils/ajax.html")
#c = c.replace("BELOW", "-->")
#c = c.replace("ABOVE", "<!--")
display(HTML(c))
c = readFile("../PyUtils/postgres.html")
#c = c.replace("BELOW", "-->")
#c = c.replace("ABOVE", "<!--")
display(HTML(c))

<IPython.core.display.Javascript object>